In [2]:
import tensorflow
from client.models.AMLmodel import AMLModel

## Create model

In [3]:
model = AMLModel()

## Load training data

In [4]:
import pickle
import numpy as np
import os
import_folder = 'dataset/raw'

x_train = pickle.load( open(import_folder + "/x_train.p", "rb" ) )
y_train = pickle.load( open(import_folder + "/y_train.p", "rb" ) )

x_val = pickle.load( open(import_folder + "/x_val.p", "rb" ) )
y_val = pickle.load( open(import_folder + "/y_val.p", "rb" ) )

x_test = pickle.load( open(import_folder + "/x_test.p", "rb" ) )
y_test = pickle.load( open(import_folder + "/y_test.p", "rb" ) )

In [5]:
def pre_process(x_data, y_data):
    num_classes = 15
    y_data = tensorflow.keras.utils.to_categorical(y_data, num_classes)
    x_data = x_data.astype('float32')
    x_data /= 255
    return x_data, y_data

In [6]:
x_train, y_train = pre_process(x_train,y_train)
x_val, y_val = pre_process(x_val,y_val)

In [7]:
x_train.shape

(15025, 100, 100, 4)

## Train model

In [ ]:
model.fit(x_train,y_train,x_val,y_val,batch_size=8, data_augmentation=False)
model_json = model.model.to_json()
f = open('model.json', 'w')
f.write(model_json)
f.close()
model.model.save_weights('model_weights.h5')

Not using data augmentation.


In [1]:
model.model = tensorflow.keras.models.load_model('MLmodel.hdf5')

NameError: name 'keras' is not defined

In [ ]:
report = model.classification_report(x_test,y_test)
print(report)

In [ ]:
report = model.classification_report(x_test,y_test)
print(report)

In [ ]:
np.argmax(y_test,1)
np.unique(np.argmax(y_test,1),return_counts=True)

In [ ]:
from matplotlib import pylab as plt
import itertools
M = model.confusion_matrix(x_test,y_test)
M = np.int32(M[:15,:15])
print(M)
print("----")
print(np.sum(M,1))
print("----")
M = (M.T/np.sum(M,1)).T

print(M)

f, ax = plt.subplots(1,1,figsize=(15,15))
ax.imshow(M,cmap='Purples')

# Major ticks
ax.set_xticks(np.arange(0, 15, 1));
ax.set_yticks(np.arange(0, 15, 1));

# Labels for major ticks
ax.set_xticklabels(np.arange(1, 15, 1));
ax.set_yticklabels(np.arange(1, 15, 1));

# Minor ticks
ax.set_xticks(np.arange(-.5, 14, 1), minor=True);
ax.set_yticks(np.arange(-.5, 14, 1), minor=True);

ax.grid(which='minor', color='black', linestyle='-', linewidth=2)


ax.set_xlabel('Predictions')
ax.set_ylabel('Truth')
upper_thresh = 0.5
format(M[i, j], '.2f')
for i,j in itertools.product(range(15),range(15)):
    ax.text(j, i, format(M[i, j], '.2f'), horizontalalignment="center", fontsize=20,
                 color="white" if (M[i, j] > upper_thresh ) else "black")

plt.show()

In [ ]:
from matplotlib import pylab as plt
import itertools
M = model.confusion_matrix(x_test,y_test)
M = np.int32(M[:15,:15])

M = (M.T/np.sum(M,1)).T


f, ax = plt.subplots(1,1,figsize=(15,15))
ax.imshow(M,cmap='Purples')

# Major ticks
ax.set_xticks(np.arange(0, 15, 1));
ax.set_yticks(np.arange(0, 15, 1));

# Labels for major ticks
ax.set_xticklabels(np.arange(1, 15, 1));
ax.set_yticklabels(np.arange(1, 15, 1));

# Minor ticks
ax.set_xticks(np.arange(-.5, 14, 1), minor=True);
ax.set_yticks(np.arange(-.5, 14, 1), minor=True);

ax.grid(which='minor', color='black', linestyle='-', linewidth=2)


ax.set_xlabel('Predictions')
ax.set_ylabel('Truth')
upper_thresh = 0.5
format(M[i, j], '.2f')
for i,j in itertools.product(range(15),range(15)):
    ax.text(j, i, format(M[i, j], '.2f'), horizontalalignment="center", fontsize=20,
                 color="white" if (M[i, j] > upper_thresh ) else "black")

plt.show()

In [ ]:
y_pred = model.model.predict_classes(x_test)
y_test_ = np.argmax(y_test,1)

In [ ]:
print(y_pred.shape)
print(y_test_.shape)

for pred_, true_ in zip(y_pred,y_test_):
    print(pred_, " - ", true_)